# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [149]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [150]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [151]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [152]:
datab = data.groupby(["CustomerID", "ProductName"]).agg({"Quantity":"sum"})
datab

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [153]:
datab2 = pd.pivot_table(datab,values="Quantity",index="ProductName", columns=["CustomerID"])
datab2 = datab2.fillna(value=0)
datab2

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [154]:
square = squareform(pdist(datab2, metric='euclidean'))
square

array([[  0.        , 279.99642855, 253.66907577, ..., 255.67166444,
        258.0910692 , 278.47800631],
       [279.99642855,   0.        , 271.49953959, ..., 265.7367118 ,
        279.55500353, 286.80655502],
       [253.66907577, 271.49953959,   0.        , ..., 245.27535547,
        253.01580978, 269.43644891],
       ...,
       [255.67166444, 265.7367118 , 245.27535547, ...,   0.        ,
        264.14200726, 268.28343221],
       [258.0910692 , 279.55500353, 253.01580978, ..., 264.14200726,
          0.        , 273.15014186],
       [278.47800631, 286.80655502, 269.43644891, ..., 268.28343221,
        273.15014186,   0.        ]])

In [155]:
distances = pd.DataFrame(1/(1 + squareform(pdist(datab2.T, 'euclidean'))), 
                         index=datab2.columns, columns=datab2.columns)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [156]:
similar = distances.loc[33]
similarb = similar.sort_values(ascending=False)[1:6]
similarb = pd.DataFrame(similarb)

In [157]:
similaro = pd.DataFrame(distances.loc[33].sort_values(ascending=False)[0:6])
similaro
#similarb = similar.sort_values(ascending=False)[0:6]

,33
CustomerID,
33,1.000000
264,0.087047
3535,0.087047
3317,0.087047
2503,0.085983
3305,0.085638


In [158]:
similarb

,33
CustomerID,
264,0.087047
3535,0.087047
3317,0.087047
2503,0.085983
3305,0.085638


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [159]:
lista = [i for i in similarb.index]

In [160]:
data5 = datab.loc[lista]
data5

Quantity
CustomerID ProductName                             
264        Apricots - Halves                      1
           Apricots Fresh                         1
           Bacardi Breezer - Tropical             1
           Bagel - Plain                          1
           Banana - Leaves                        1
...                                             ...
3535       Tray - 16in Rnd Blk                    1
           Wanton Wrap                            1
           Wine - Blue Nun Qualitatswein          1
           Wine - Magnotta, Merlot Sr Vqa         1
           Yoghurt Tubes                          1

[270 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [161]:
data6 = data5.groupby(["ProductName"]).agg({"Quantity":"sum"}).sort_values("Quantity",ascending=False)
data6

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
...,...
Hinge W Undercut,1
Ice Cream Bar - Hageen Daz To,1
Jagermeister,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [162]:
data7 = pd.DataFrame(datab2[33]).reset_index()
data7 = data7[data7[33] != 0]
data7.head()

,ProductName,33
4,Apricots - Dried,1.0
9,Assorted Desserts,1.0
17,Bandage - Flexible Neon,1.0
19,"Bar Mix - Pina Colada, 355 Ml",1.0
23,"Beans - Kidney, Canned",1.0


In [163]:
recomendados = [i for i in data6.index if i not in data7.ProductName]
recomendados[0:5]

['Butter - Unsalted',
 'Wine - Ej Gallo Sierra Valley',
 'Towels - Paper / Kraft',
 'Soup - Campbells Bean Medley',
 'Wine - Blue Nun Qualitatswein']

In [164]:
"""def iterador():
    i,j=0,0
    while i<5:
        if data6.index[j] not in data7.ProductName:
            yield data6.index[j]
            i+=1
        j+=1
        if j == len(data6.index): break
            
rec = [i for i in iterador()]
"""

'def iterador():\n    i,j=0,0\n    while i<5:\n        if data6.index[j] not in data7.ProductName:\n            yield data6.index[j]\n            i+=1\n        j+=1\n        if j == len(data6.index): break\n            \nrec = [i for i in iterador()]\n'

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [165]:
customerid = [i for i in data.CustomerID][0:50]

In [166]:
def recomendaciones(lista):
    recom = {}
    for a in lista:
        similaro = pd.DataFrame(distances.loc[a].sort_values(ascending=False)[0:6])
        similaro = similaro[1:6]
        lista = [i for i in similaro.index]
        data5 = datab.loc[lista]
        data6 = data5.groupby(["ProductName"]).agg({"Quantity":"sum"}).sort_values("Quantity",ascending=False)
        data7 = pd.DataFrame(datab2[a]).reset_index()
        data7 = data7[data7[a] != 0]
        recomendados = [i for i in data6.index if i not in data7.ProductName]
        recomendados = recomendados[0:5]
        recom[a] = recomendados
    return recom

In [168]:
recomendaciones_todos = recomendaciones(customerid)

La función funciona, pero si le pones una lista de customers muy larga..... tarda!

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [171]:
df9 = pd.DataFrame(recomendaciones_todos)
df9

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,55162,22345,81876,48641,81034,29681,87947,28867,69967,94420
0,Beef - Rib Eye Aaa,Wine - Valpolicella Masi,Bread - Italian Corn Meal Poly,Tea - English Breakfast,Isomalt,Cocoa Butter,Juice - Lime,Vinegar - Sherry,Soup Knorr Chili With Beans,Sugar - Fine,...,Wasabi Powder,Ice Cream Bar - Oreo Cone,Ocean Spray - Ruby Red,Guinea Fowl,Juice - Happy Planet,Bread Foccacia Whole,Island Oasis - Mango Daiquiri,Guinea Fowl,Rambutan,Juice - V8 Splash
1,Jagermeister,Fuji Apples,"Wine - White, Colubia Cresh",Squid - Tubes / Tenticles 10/20,Crush - Cream Soda,"Cheese - Boursin, Garlic / Herbs","Pork - Bacon, Double Smoked",Bread Foccacia Whole,Soupfoamcont12oz 112con,Cheese - Mozzarella,...,"Nut - Chestnuts, Whole",Wine - Valpolicella Masi,Cheese - Taleggio D.o.p.,Phyllo Dough,Banana - Leaves,Lamb - Ground,"Fish - Scallops, Cold Smoked","Nut - Chestnuts, Whole",Wine - Valpolicella Masi,Yoghurt Tubes
2,Chicken - Soup Base,Tahini Paste,Puree - Mocha,Spinach - Baby,Vanilla Beans,Cake - Mini Cheesecake,Tomatoes Tear Drop,Wine - Blue Nun Qualitatswein,Veal - Sweetbread,Bananas,...,Bread Crumbs - Panko,Olives - Kalamata,Yogurt - French Vanilla,Initation Crab Meat,Apricots Fresh,"Salsify, Organic",Spice - Peppercorn Melange,"Soup - Canadian Pea, Dry Mix",Bread - Italian Corn Meal Poly,Duck - Breast
3,Lime Cordial - Roses,Ocean Spray - Ruby Red,"Cheese - Brie,danish",Jolt Cola - Electric Blue,Cassis,Wiberg Super Cure,Pomello,Bread - French Baquette,"Wine - Magnotta, Merlot Sr Vqa",Quiche Assorted,...,Halibut - Steaks,Juice - Apple Cider,"Lemonade - Natural, 591 Ml",Oil - Safflower,Pastry - Butterscotch Baked,Wine - Two Oceans Cabernet,Chocolate - Dark,Ezy Change Mophandle,Pop Shoppe Cream Soda,Brandy - Bar
4,"Lamb - Pieces, Diced",Tuna - Salad Premix,Cookie - Dough Variety,Sprouts - Baby Pea Tendrils,Tea - Earl Grey,Beef - Top Sirloin - Aaa,Island Oasis - Mango Daiquiri,Table Cloth 62x114 White,Cocktail Napkin Blue,"Pork - Loin, Center Cut",...,Chips Potato All Dressed - 43g,Pate - Cognac,Olive - Spread Tapenade,French Pastry - Mini Chocolate,Beets - Mini Golden,Vinegar - Sherry,"Veal - Brisket, Provimi,bnls",Beer - Sleemans Cream Ale,Table Cloth 54x72 White,Wine - Redchard Merritt


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [191]:
distances2 = pd.DataFrame(1/(1 + squareform(pdist(datab2.T, 'cityblock'))), 
                         index=datab2.columns, columns=datab2.columns)

In [192]:
def recomendaciones(lista):
    recom = {}
    for a in lista:
        similaro = pd.DataFrame(distances2.loc[a].sort_values(ascending=False)[0:6])
        similaro = similaro[1:6]
        lista = [i for i in similaro.index]
        data5 = datab.loc[lista]
        data6 = data5.groupby(["ProductName"]).agg({"Quantity":"sum"}).sort_values("Quantity",ascending=False)
        data7 = pd.DataFrame(datab2[a]).reset_index()
        data7 = data7[data7[a] != 0]
        recomendados = [i for i in data6.index if i not in data7.ProductName]
        recomendados = recomendados[0:5]
        recom[a] = recomendados
    return recom

In [193]:
recomendaciones_todos2 = recomendaciones(customerid)

In [194]:
df10 = pd.DataFrame(recomendaciones_todos2)
df10

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,55162,22345,81876,48641,81034,29681,87947,28867,69967,94420
0,Soup - Campbells Bean Medley,Butter - Unsalted,"Pepper - Black, Whole",Beef - Texas Style Burger,Veal - Sweetbread,Butter - Unsalted,Juice - Lime,Soup - Campbells Tomato Ravioli,Butter - Unsalted,Knife Plastic - White,...,Beef Wellington,"Pepper - Black, Whole",Pork - Kidney,Pork - Kidney,Cod - Black Whole Fillet,"Cheese - Boursin, Garlic / Herbs",Yoghurt Tubes,Wanton Wrap,Butter - Unsalted,Anchovy Paste - 56 G Tube
1,Butter - Unsalted,Tia Maria,Knife Plastic - White,"Wine - White, Colubia Cresh",Knife Plastic - White,Tia Maria,"Pork - Bacon, Double Smoked",Pork - Kidney,Knife Plastic - White,Wine - Toasted Head,...,Flour - Pastry,"Bar Mix - Pina Colada, 355 Ml","Sole - Dover, Whole, Fresh",Olives - Stuffed,Curry Paste - Madras,Chinese Foods - Chicken,Halibut - Steaks,Ice Cream Bar - Hageen Daz To,"Soup - Campbells, Lentil",Halibut - Steaks
2,"Oranges - Navel, 72",Wine - Blue Nun Qualitatswein,Butter - Unsalted,Pork - Kidney,Cheese - Mozzarella,Veal - Sweetbread,Bar - Granola Trail Mix Fruit Nut,Cup - Translucent 7 Oz Clear,"Pepper - Black, Whole","Wine - Magnotta, Merlot Sr Vqa",...,"Pasta - Penne, Rigate, Dry",Garlic - Elephant,Cup - Translucent 7 Oz Clear,"Pasta - Detalini, White, Fresh",Cheese - Mozzarella,Pork - Kidney,Muffin - Carrot Individual Wrap,Pork - Kidney,Bay Leaf,Cheese - Cambozola
3,Wine - Ej Gallo Sierra Valley,Juice - Lime,Wine - Ej Gallo Sierra Valley,Knife Plastic - White,Muffin - Carrot Individual Wrap,Knife Plastic - White,Tomatoes Tear Drop,Bread Crumbs - Panko,Soup - Campbells Bean Medley,Chinese Foods - Chicken,...,Tia Maria,Muffin Batt - Blueberry Passion,Soup - Campbells Tomato Ravioli,Wine - Blue Nun Qualitatswein,Ocean Spray - Ruby Red,Pastry - Cheese Baked Scones,Coffee - Irish Cream,"Wine - Magnotta, Merlot Sr Vqa","Tart Shells - Sweet, 4",Cheese - Wine
4,Bay Leaf,"Sole - Dover, Whole, Fresh",Wine - Blue Nun Qualitatswein,Squid - Tubes / Tenticles 10/20,Quiche Assorted,Beef - Montreal Smoked Brisket,"Pepper - Paprika, Hungarian",Lime Cordial - Roses,Cod - Black Whole Fillet,Wine - Blue Nun Qualitatswein,...,Veal - Sweetbread,Macaroons - Two Bite Choc,Foam Dinner Plate,Lamb - Ground,Beans - Wax,Cheese - Wine,Pork - Kidney,Berry Brulee,"Nut - Chestnuts, Whole",Tray - 16in Rnd Blk


In [195]:
print(df9 == df10)

   61288  77352  40094  23548  78981  83106  11253  35107  15088  26031  ...  \
0  False  False  False  False  False  False  False  False  False  False  ...   
1  False  False  False  False  False  False  False  False  False  False  ...   
2  False  False  False  False  False  False  False  False  False  False  ...   
3  False  False  False  False  False  False  False  False  False  False  ...   
4  False  False  False  False  False  False  False  False  False  False  ...   

   55162  22345  81876  48641  81034  29681  87947  28867  69967  94420  
0  False  False  False  False  False  False  False  False  False  False  
1  False  False  False  False  False  False  False  False  False  False  
2  False  False  False  False  False  False  False  False  False  False  
3  False  False  False  False  False  False  False  False  False  False  
4  False  False  False  False  False  False  False  False  False  False  

[5 rows x 49 columns]
